In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from parse import parse

# Initializing driver
driver = webdriver.Firefox()
driver.get("https://coursefeedback.uchicago.edu/?Department=CMSC&AcademicYear=2022&AcademicTerm=Spring")
time.sleep(35)

# Enter your credientials here

Questions = {}

In [2]:
def get_links(driver, dep, year, term):
# Get all courses in the search
    feedback_link = "https://coursefeedback.uchicago.edu/?Department={}&\
AcademicYear={}&AcademicTerm={}".format(dep, year, term)
    driver.get(feedback_link)
    courses = driver.find_elements(By.CSS_SELECTOR, "td[class='course']>a")

    links = []
    for course in courses:
        links.append(course.get_attribute("href"))

    return links

In [5]:
def get_id(link):
    id = parse("https://uchicago.bluera.com/uchicago/rpvf-eng.aspx?lang=eng&redi=1&SelectedIDforPrint={}&ReportType=2&regl=en-US", link)

    return id[0]

def parse_catalog(catalog_num):
    result = {}
    subject, course_id, section = catalog_num.strip().split(' ')
    result["Subject"] = subject
    result["CourseId"] = course_id
    result["Section"] = section

    return result
    


In [15]:
def get_eval(driver, link, id):
    # Get one evaluation 
    evaluation = {}
    evaluation['Id'] = id
    driver.get(link)
    time.sleep(1)

    # Get eval metadata
    report_metadata = driver.find_element(By.CLASS_NAME, "metadata")

    report_title = report_metadata.find_element(By.TAG_NAME, "h2").text
    evaluation["Report Title"] = report_title
    course_num, course_name, instructors = parse("{} - {} - Instructor(s){}", report_title)
    evaluation["Catalog Number"] = [parse_catalog(num) for num in course_num.split(",")]
    evaluation["Course Name"] = course_name
    evaluation["Instructors"] = [instructor.strip(" :") for instructor in instructors.split(",")]

    offer_time, num_enrolled, num_respond = report_metadata.find_elements(By.TAG_NAME, "dd")[:3]
    term, year = offer_time.text.split(" - ")[1].split(" ")
    evaluation["Academic Year"] = term.strip()
    evaluation["Academic Term"] = year.strip()
    evaluation["Number Enrolled"] = num_enrolled.text.strip()
    evaluation["Number Responses"] = num_respond.text.strip()

    # get the questions and answers
    text_data = {}
    reports = driver.find_elements(By.CLASS_NAME, "report-block")
    for report in reports:
        question = report.find_element(By.TAG_NAME, "span").text

        # filter out non-comment
        report_head = report.find_elements(By.TAG_NAME, "thead")
        if (len(report_head) == 0 or len(report_head[0].find_elements(By.TAG_NAME, "th")) != 1):
            continue

        #extract comments
        report_body = report.find_element(By.TAG_NAME, "tbody")
        comments = []
        report_comments = report_body.find_elements(By.TAG_NAME, "tr")
        if (len(report_comments)) == 0:
            continue

        if question not in Questions:
            Questions[question] = False
        
        for comment in report_comments[1:]:
            comments.append(comment.text)

        text_data[question] = comments

        evaluation["Evaluation Text Data"] = text_data

        evaluation["url"] = link

    return evaluation


In [7]:
import json

def scrape_site(driver, links, evals):
    for link in links:
        id = get_id(link)
        eval = get_eval(driver, link, id)
        evals[id] = eval


In [16]:
new = {}
scrape_site(driver, ["https://uchicago.bluera.com/uchicago/rpvf-eng.aspx?lang=eng&redi=1&SelectedIDforPrint=7f6c8819dcb80bc4fed16e3a97122ef7e9334992664ea293262832e79a9f2c419cd4e38ffa15dd4a406e973272af102e&ReportType=2&regl=en-US"],new)
new


{'7f6c8819dcb80bc4fed16e3a97122ef7e9334992664ea293262832e79a9f2c419cd4e38ffa15dd4a406e973272af102e': {'Id': '7f6c8819dcb80bc4fed16e3a97122ef7e9334992664ea293262832e79a9f2c419cd4e38ffa15dd4a406e973272af102e',
  'Report Title': 'MATH 27700 50, CMSC 27700 50 - Mathematical Logic I - Instructor(s): Benedict Morrissey',
  'Catalog Number': [{'Subject': 'MATH', 'CourseId': '27700', 'Section': '50'},
   {'Subject': 'CMSC', 'CourseId': '27700', 'Section': '50'}],
  'Course Name': 'Mathematical Logic I',
  'Instructors': ['Benedict Morrissey'],
  'Academic Year': 'Autumn',
  'Academic Term': '2022',
  'Number Enrolled': '29',
  'Number Responses': '13',
  'Evaluation Text Data': {'What are the most important things that you learned in this course? Please reflect on the knowledge and skills you gained.': ['Model theory, propositional and first–order logic.',
    'Cardinality, propositional logic, first order logic',
    'Model Theory, First order logic, and propositional logic.',
    'How to app

In [17]:
# Scrape one year for all department
from majors import majors
quarters = ['Summer', 'Autumn', 'Winter', 'Spring']

def get_whole_year(driver, year):
    evals = {}

    for major in majors:
        links = get_links(driver, major, year, 'All')
        time.sleep(5)
        if (len(links) == 0):
            continue
        scrape_site(driver, links, evals)

    f_name = "data2/" + str(year) + ".json"
    with open(f_name, "w") as f:
        json.dump(evals, f)

    return evals

get_whole_year(driver, 2022)
q_name = "q_type_2022.py"
with open(q_name, "w") as f:
    json.dump(Questions, f) 

Questions = {}
get_whole_year(driver, 2021)
q_name = "q_type_2021.py"
with open(q_name, "w") as f:
    json.dump(Questions, f) 


KeyboardInterrupt: 

{'What are the most important things that you learned in this course? Please reflect on the knowledge and skills you gained.': False, 'Describe how aspects of this course (lectures, discussions, labs, assignments, etc.) contributed to your learning.': False, 'Additional comments about the course:': False, "Thinking about your time in the class, what aspect of the instructor's teaching contributed most to your learning?": False, 'What could the instructor modify to help you learn more?': False, "Please include the name of the TA/CA/Intern you are evaluating. What aspects of the TA's teaching contributed most to your learning? What could the TA modify to help you learn more? Please include any additional feedback for the TA/CA/Intern.": False, 'Other course elements not mentioned above:': False, 'Please comment on the level of difficulty of the course relative to your background and experience.': False, "What aspect of the instructor's teaching contributed most to your learning?": False,